<a href="https://colab.research.google.com/github/ovsienkobohdan/arrhythmia-detection/blob/main/model_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!wget -r -N -c -np https://physionet.org/files/mitdb/1.0.0/

In [ ]:
!pip install wfdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import wfdb
from tqdm import tqdm

# Data

In [3]:
data_path = '/content/physionet.org/files/mitdb/1.0.0/'

In [4]:
patients_file = "RECORDS"
with open(data_path+patients_file) as f:
    patients = f.read().splitlines()
print(patients)

['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']


In [81]:
def load_ecg(file, channels):
    record = wfdb.rdrecord(file, channels=channels)
    annotation = wfdb.rdann(file, 'atr')
    
    p_signal = record.p_signal
    
    assert record.fs == 360, 'sample freq is not 360'
    
    atr_sym = annotation.symbol
    atr_sample = annotation.sample

    return p_signal, atr_sym, atr_sample, aux_note

In [6]:
def beat_annotations(annotations, annotations_sample, beat):
  inds = np.in1d(annotations, beat)
  beats = annotations_sample[inds]
  return beats

In [68]:
# TODO: remove keys with empty values

def segmentation(data_path, patients, channels=[0]):
  patient_beats = {}
  
  for patient in tqdm(patients):
    beats = {"N":[], "L":[], "R":[], "A":[], "V":[], "/":[], "E":[]}
    file_path = data_path + patient
    signal, atr_sym, atr_sample = load_ecg(file_path, channels=channels)

    for b in beats:
      type_beats = beat_annotations(annotations=atr_sym, annotations_sample=atr_sample, beat=b)
      
      for type_beat in type_beats:
        all_beats_ind = list(atr_sample).index(type_beat)

        if all_beats_ind!=0 and all_beats_ind!=(len(atr_sample)-1):
          prev_beat = atr_sample[all_beats_ind-1]
          next_beat = atr_sample[all_beats_ind+1]
          left_offset = abs(prev_beat - type_beat)//2
          right_offset = abs(next_beat - type_beat)//2
          beats[b].append(signal[type_beat - left_offset:type_beat+right_offset,0])

    patient_beats[patient] = beats
  return patient_beats

In [69]:
B = segmentation(data_path, patients)

100%|██████████| 48/48 [00:31<00:00,  1.53it/s]
